# Prompt Steering vs Activation Steering in LLMs

This notebook compares three ways of controlling LLM behavior:

1. Baseline generation (no steering)
2. Prompt-based steering
3. Activation steering (hidden-state manipulation)

We focus on a single behavioral axis:  
**Confidence vs Hedging under prompt conflict**.

The goal is to demonstrate where prompt steering becomes brittle and how activation steering can offer more consistent control at inference time.


## Model Choice and Setup

We use a small, open-weight transformer model from Hugging Face to ensure:

- Full access to hidden states
- Inference-time manipulation
- Compatibility with Google Colab

No fine-tuning or weight updates are performed.
All experiments use the same frozen model.


In [ ]:
!pip install -q transformers accelerate

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
from huggingface_hub import login
from google.colab import userdata

try:
    # Attempt to log in using the 'HF_TOKEN' secret
    login(token=userdata.get('HF_TOKEN'))
except Exception:
    # Fallback to interactive login if secret is missing
    print("Secret 'HF_TOKEN' not found. Please log in interactively.")
    login()

In [ ]:
model_name = "google/gemma-3-4b-it"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    dtype=torch.float32,
    device_map="auto",
    output_hidden_states=True
)

model.eval()

## Task Definition

Base prompt used for all comparisons:

"Explain whether AI will replace software engineers."

We compare:
- Baseline behavior
- Prompt-based confidence steering
- Activation-based confidence steering

Behavior is evaluated using a composite Confidence Score that measures:
- Assertive language
- Hedge markers
- Contrast/conditional markers

This allows to quantify confidence rather than relying on single keywords.

In [ ]:
def generate_text(prompt, max_new_tokens=120, seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.7
        )

    return tokenizer.decode(output[0], skip_special_tokens=True)

In [ ]:
base_prompt = "Explain whether AI will replace software engineers."

print("=== BASELINE ===")
print(generate_text(base_prompt))

## Prompt-Based Steering

We explicitly instruct the model to be confident and decisive.
This approach relies entirely on input text, without modifying internal activations.

In [ ]:
prompt_steered = """
You are a confident and decisive expert.
Do not hedge or express uncertainty.

Explain whether AI will replace software engineers.
"""

print("=== PROMPT STEERED ===")
print(generate_text(prompt_steered))

## Activation Steering

Instead of steering via text, we steer the model internally.

We construct a steering vector by contrasting:
- Confident statements
- Hedging statements

This vector is injected into a middle transformer layer during inference.


In [ ]:
def get_layer_hidden_state(prompt, layer_idx=12):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        # Explicitly request hidden states here
        outputs = model(**inputs, output_hidden_states=True)

    # hidden_states: tuple (layer, batch, seq, hidden)
    hidden = outputs.hidden_states[layer_idx]
    return hidden.mean(dim=1)  # mean over tokens

In [ ]:
confident_prompts = [
    "AI will definitely transform software engineering.",
    "Software engineers will remain essential and in control.",
    "AI is a powerful tool, not a replacement."
]

hedging_prompts = [
    "AI might replace some software engineers.",
    "It depends on many factors.",
    "AI could possibly change engineering roles."
]

conf_states = torch.stack([get_layer_hidden_state(p) for p in confident_prompts])
hedge_states = torch.stack([get_layer_hidden_state(p) for p in hedging_prompts])

steering_vector = conf_states.mean(dim=0) - hedge_states.mean(dim=0)

We now inject the steering vector during generation using a forward hook.
The prompt remains neutral.


In [ ]:
layer_to_steer = 12
alpha = 1.5  # steering strength

def steering_hook(module, input, output):
    # Transformer layers usually return a tuple (hidden_states, past_key_values, ...)
    if isinstance(output, tuple):
        hidden_states = output[0]
        # steering_vector is (1, hidden_dim). unsqueeze(1) -> (1, 1, hidden_dim)
        modified = hidden_states + alpha * steering_vector.unsqueeze(1)
        return (modified,) + output[1:]
    else:
        # Fallback if output is just a tensor
        return output + alpha * steering_vector.unsqueeze(1)

# Correct path for Gemma 3: model.model.language_model.layers
handle = model.model.language_model.layers[layer_to_steer].register_forward_hook(steering_hook)

print("=== ACTIVATION STEERED ===")
print(generate_text(base_prompt))

handle.remove()

## Initial Comparison

We compare outputs generated from the same base prompt:

- Baseline: balanced, instruction-aligned behavior
- Prompt-steered: increased confidence but sampling variability
- Activation-steered: comparable confidence with lower variance

This preliminary comparison motivates deeper statistical evaluation.


## Behavioral Metric

We compute a composite Confidence Score based on:

+ Assertive markers
- Hedge markers
- Contrast/conditional markers

Higher score indicates stronger commitment and lower hedging.


In [ ]:
hedge_markers = [
    " may ", " might ", " could ", " likely ", " possibly ",
    " depends ", " uncertain ", " unclear ",
    " unlikely ", " in some cases ", " in many cases ",
    " complex ", " nuanced "
]

contrast_markers = [
    " however ", " but ", " although ", " while ",
    " on the other hand ", " it depends "
]

assertive_markers = [
    " will ", " is ", " definitely ",
    " inevitable ", " clearly ", " certainly "
]

def confidence_score(text):
    text = text.lower()

    hedge = sum(text.count(w) for w in hedge_markers)
    contrast = sum(text.count(w) for w in contrast_markers)
    assertive = sum(text.count(w) for w in assertive_markers)

    score = assertive - hedge - contrast

    return {
        "assertive": assertive,
        "hedge": hedge,
        "contrast": contrast,
        "confidence_score": score
    }


In [ ]:
baseline_out = generate_text(base_prompt)
prompt_out = generate_text(prompt_steered)

handle = model.model.language_model.layers[layer_to_steer].register_forward_hook(steering_hook)
activation_out = generate_text(base_prompt)
handle.remove()

print("Baseline:", confidence_score(baseline_out))
print("Prompt-Steered:", confidence_score(prompt_out))
print("Activation-Steered:", confidence_score(activation_out))

## Statistical Confidence Comparison

Across multiple sampled generations:

- Both prompt and activation steering increase mean confidence relative to baseline.
- Prompt steering exhibits higher variance.
- Activation steering achieves comparable mean confidence with lower variance.

This suggests activation steering provides more stable behavioral control.


In [ ]:
import numpy as np

def evaluate_mode(prompt, apply_activation=False, n_runs=5):
    scores = []

    for i in range(n_runs):
        if apply_activation:
            handle = model.model.language_model.layers[layer_to_steer].register_forward_hook(steering_hook)
            output = generate_text(prompt, seed=42 + i)
            handle.remove()
        else:
            output = generate_text(prompt, seed=42 + i)

        score_dict = confidence_score(output)
        scores.append(score_dict["confidence_score"])

    return np.mean(scores), np.std(scores)


In [ ]:
base_mean, base_std = evaluate_mode(base_prompt, apply_activation=False)
prompt_mean, prompt_std = evaluate_mode(prompt_steered, apply_activation=False)
act_mean, act_std = evaluate_mode(base_prompt, apply_activation=True)

print("Baseline:", base_mean, "±", base_std)
print("Prompt-Steered:", prompt_mean, "±", prompt_std)
print("Activation-Steered:", act_mean, "±", act_std)


In [ ]:
import matplotlib.pyplot as plt

modes = ["Baseline", "Prompt", "Activation"]
means = [base_mean, prompt_mean, act_mean]
stds = [base_std, prompt_std, act_std]

plt.figure()
plt.bar(modes, means)
plt.title("Confidence Score Comparison")
plt.ylabel("Confidence Score")
plt.show()

The visualization highlights that while mean confidence scores are similar between prompt and activation steering, activation steering demonstrates reduced variability.

This supports the hypothesis that modifying internal representations yields more stable behavior than relying solely on prompt instructions.


## Prompt Stress Test

We now introduce conflicting instructions to the prompt.

This tests whether steering survives when the prompt pushes
the model toward uncertainty.


In [ ]:
stress_prompt = """
Explain whether AI will replace software engineers.
Be careful, acknowledge uncertainty, and consider multiple perspectives.
"""

In [ ]:
print("=== PROMPT STEERED (STRESS) ===")
print(generate_text(prompt_steered + "\n\n" + stress_prompt))

handle = model.model.language_model.layers[layer_to_steer].register_forward_hook(steering_hook)
print("\n=== ACTIVATION STEERED (STRESS) ===")
print(generate_text(stress_prompt))
handle.remove()

## Stress Test Averaging

We evaluate steering stability under conflicting instructions.

This tests whether activation steering maintains confidence
when the prompt explicitly asks for uncertainty.


In [ ]:
# Evaluate prompt-steered under stress
stress_prompt_combined = prompt_steered + "\n\n" + stress_prompt

prompt_stress_mean, prompt_stress_std = evaluate_mode(
    stress_prompt_combined,
    apply_activation=False
)

# Evaluate activation steering under stress
act_stress_mean, act_stress_std = evaluate_mode(
    stress_prompt,
    apply_activation=True
)

print("Prompt-Steered (Stress):", prompt_stress_mean, "±", prompt_stress_std)
print("Activation-Steered (Stress):", act_stress_mean, "±", act_stress_std)


In [ ]:
modes = ["Prompt-Stress", "Activation-Stress"]
means = [prompt_stress_mean, act_stress_mean]
stds = [prompt_stress_std, act_stress_std]

plt.figure()
plt.bar(modes, means)
plt.title("Confidence Under Prompt Conflict")
plt.ylabel("Confidence Score")
plt.show()


## Ablation Study

We test:
- Different transformer layers
- Different steering strengths (alpha)

This checks whether activation steering is robust
or dependent on a single configuration.


## Layer Sensitivity Analysis

Steering effects vary substantially by layer.

- Early-middle layers (e.g., Layer 6) produce large shifts in confidence score.
- Later layers show reduced behavioral impact.

However, extreme early-layer steering introduces structural degeneration,
revealing a tradeoff between behavioral strength and coherence.


In [ ]:
layer_results = {}

for layer in [6, 12, 18]:
    layer_to_steer = layer
    mean_score, std_score = evaluate_mode(base_prompt, apply_activation=True)
    layer_results[layer] = mean_score
    print(f"Layer {layer} | Mean Confidence: {mean_score}")


In [ ]:
plt.figure()
plt.plot(list(layer_results.keys()), list(layer_results.values()))
plt.title("Layer vs Confidence Score")
plt.xlabel("Layer")
plt.ylabel("Confidence Score")
plt.show()

### Coherence Check: Extreme Steering Layer

Layer 6 produced unusually high confidence scores.
We manually inspect multiple generations to verify coherence.

In [ ]:
# Inspect 3 samples from layer 6 steering

layer_to_steer = 6

for i in range(3):
    handle = model.model.language_model.layers[layer_to_steer].register_forward_hook(steering_hook)
    output = generate_text(base_prompt, seed=100 + i)
    handle.remove()

    print(f"\n--- Sample {i+1} ---\n")
    print(output)


In [ ]:
def length_stats(prompt, apply_activation=False):
    lengths = []
    for i in range(5):
        if apply_activation:
            handle = model.model.language_model.layers[layer_to_steer].register_forward_hook(steering_hook)
            output = generate_text(prompt, seed=200 + i)
            handle.remove()
        else:
            output = generate_text(prompt, seed=200 + i)

        lengths.append(len(output.split()))

    return np.mean(lengths)

print("Baseline length:", length_stats(base_prompt, apply_activation=False))
print("Layer 6 length:", length_stats(base_prompt, apply_activation=True))


## Observation: Steering–Coherence Tradeoff

Early-layer steering (e.g., Layer 6) produces very high confidence scores,
but introduces structural degeneration and repetition.

This suggests a tradeoff:
- Stronger activation shifts increase behavioral signal
- But can degrade generation quality

Middle layers provide a better balance between control and coherence.


## Steering Strength (Alpha) Analysis

Increasing steering strength does not monotonically increase confidence.

Higher alpha values can reduce confidence score or introduce instability,
indicating oversteering may distort internal representations.

This suggests the existence of an optimal steering magnitude
that balances behavioral shift and coherence.


In [ ]:
alpha_results = {}

for alpha in [0.5, 1.0, 1.5, 2.0]:
    # Define a robust hook that captures the current alpha
    def steering_hook(module, input, output, alpha_val=alpha):
        if isinstance(output, tuple):
            return (output[0] + alpha_val * steering_vector.unsqueeze(1),) + output[1:]
        else:
            return output + alpha_val * steering_vector.unsqueeze(1)

    mean_score, std_score = evaluate_mode(base_prompt, apply_activation=True)
    alpha_results[alpha] = mean_score
    print(f"Alpha {alpha} | Mean Confidence: {mean_score}")

In [ ]:
plt.figure()
plt.plot(list(alpha_results.keys()), list(alpha_results.values()))
plt.title("Alpha vs Confidence Score")
plt.xlabel("Alpha")
plt.ylabel("Confidence Score")
plt.show()

## Discussion

This experiment demonstrates:

- Prompt steering increases confidence but exhibits higher sampling variability.
- Activation steering produces comparable mean confidence with improved stability.
- Steering effects are strongly layer-dependent.
- Early-layer steering amplifies behavioral shifts but can degrade coherence.
- Oversteering reduces quality, revealing a control–coherence tradeoff.

In instruction-tuned models, activation steering does not override alignment,
but it provides a lightweight and interpretable inference-time biasing mechanism.

Future work may explore:
- Base (non-instruction-tuned) models
- Alternative behavioral axes
- Stronger coherence and repetition metrics
- Cross-prompt generalization
